# Data cleaning task

In this task, we will clean the data corpus based on couple of assumptions.  First of all, we'll run an off-the-shelf LID system to dump any sentences that are most likely OOL (out of language).  Then we will do some heuristic pruning, such as maximum sentence length (in words), amount of balast (non-sense characters, numbers) and finally we want to train some classifier that hopefully filters out the data based on some training corpus.  According to the WMT18 Parallel Corpus Filtering evaluation plan, it is possible to use "WMT 2018 news translation task data for German-English (without the Paracrawl parallel corpus) to train components of your method".  Without much studies, I chose the Europarl de-en set.

While working on this, I found that LID takes a lot of time, so I let the system run while working on other stuff and we load the noisy corpus right with the LID data.

In [ ]:
import json
import string
import numpy as np
import mymt

# THis is the original file to work on.  For sake of speedup, let us use the
# file augmented with LID info.  If you want to generate it, please run the run_lid.py script
# noisy_corpus_file   = 'assignment/1-data-cleaning/noisy-corpus.json'
NOISY_CORPUS_FILE   = 'noisy-corpus.with_lid.json'

NONSENSE_CHARS      = string.punctuation + '0123456789'
MAX_SRC_WORD_COUNT  = 15
MAX_TGT_WORD_COUNT  = 15
MAX_NONSENSE_STATS  = 0.2

Let us define some helping functions

In [ ]:
# returns a proportion of non-sense characters in a string
def nonsense_stats(in_string, nonsense_chars=NONSENSE_CHARS):
    orig_len = len(in_string)
    new_len  = len(in_string.translate(str.maketrans('', '', nonsense_chars)))

    return 1.0-float(new_len)/float(orig_len)



## Basic 'common sense' filtering

Here we deploy some intuitive human knowledge base.  We'll filter out the sentences by some known vocabulary (gathered from the Europarl data, as discussed later), we'll apply a simple LID system, we'll get rid of very noisy sentences (too many punct marks and/or numbers) and we'll get rid of long sentences.

In [ ]:

print("Loading corpus {}".format(NOISY_CORPUS_FILE))
with open(NOISY_CORPUS_FILE, 'rt') as nf:
    corpus = [[ data['source'], data['target'], data['source_lid'], data['target_lid']] for data in [json.loads(line) for line in nf]]

print("Loaded {} pairs".format(len(corpus)))

print("Filtering source vocab")
corpus = [p for p in corpus if all([mymt.input_lang.isWordInVocab(w) for w in p[0].split()])]
print("Left with {} pairs".format(len(corpus)))

print("Filtering LID")
corpus = [p for p in corpus if p[2]=='de' and p[3]=='en']
print("Left with {} pairs".format(len(corpus)))

print("Filtering nonsense")
corpus = [p for p in corpus if nonsense_stats(p[0]) < MAX_NONSENSE_STATS or nonsense_stats(p[1]) < MAX_NONSENSE_STATS]
print("Left with {} pairs".format(len(corpus)))

print("Filtering very long sentences")
corpus = [p for p in corpus if len(p[0].split()) < MAX_SRC_WORD_COUNT and len(p[1].split()) < MAX_TGT_WORD_COUNT]
print("Left with {} pairs".format(len(corpus)))

## ML-based filtering

Now we'd like to do some fancy stuff.  Let us use the europarl corpus as training data for a binary classifier (accept/reject).  Europarl seems to be clean, so let us fabricate some impostors by randomizing the order of the pair mapping.  For each pair, we can extract some features and train a simple binary classifier (logistic regression).  But first, let's train a simple MT system on that data, which will help us extrac the feats.

### MT model training

In [ ]:
hidden_size = 256
encoder1 = mymt.EncoderRNN(mymt.input_lang.n_words, hidden_size).to(mymt.device)

attn_decoder1 = mymt.AttnDecoderRNN(hidden_size, mymt.output_lang.n_words, dropout_p=0.1).to(mymt.device)

mymt.trainIters(encoder1, attn_decoder1, 500, print_every=100)

OK, we should have a DE->EN model now.  Let us now use it to compute some feats.  We will use the model to translate DE to EN and compare the translation with the defined string.  Out of this, we compute the BLEU score which will serve us as one of the features.  Another feature will be the absolute length difference between the DE and EN sentence.


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction


def compute_feats(corpus, smoothing_function=SmoothingFunction().method1):
    # n-gram smoothing function
    # Default is method0 = no smoothing

    translated_sentences = [ ' '.join(mymt.evaluate(encoder1, attn_decoder1, mymt.normalizeString(p[0]))[0]) for p in corpus ]
    reference_sentences  = [ p[1] for p in corpus ]

    sent_bleu   = np.array([ sentence_bleu([ p[0].split() ], p[1].split(), smoothing_function = smoothing_function) for p in zip(reference_sentences, translated_sentences) ]).reshape((-1,1))
    sent_nwords = np.array([ abs( len(p[0]) - len(p[1]) ) for p in corpus ]).reshape((-1,1))


    return np.hstack((sent_bleu, sent_nwords))




Let us now read the classifier training data (again) which on its own will serve as the target data (positive class) and fabricate some impostors (negative class) by keeping the DE data in order, but shuffling the EN data.  We assume this is good enough for the moment.

In [ ]:
import random

foo, foo, corpus_train_orig = mymt.readLangs('de', 'en')
corpus_train_orig           = mymt.filterPairs(corpus_train_orig)

# This shuffles the EN data, but keeps the DE data inplace
corpus_train_shuffled = [q for q in zip([p[0] for p in corpus_train_orig], random.sample([e[1] for e in corpus_train_orig], len(corpus_train_orig))) ]


Training data is loaded.  Now, let's extract the feats.

In [ ]:
corpus_feats                = compute_feats(corpus)
corpus_train_orig_feats     = compute_feats(corpus_train_orig)
corpus_train_shuffled_feats = compute_feats(corpus_train_shuffled)

Feats have been extracted, time to concatenate the train feats (positive and negative portions), fabricate the labels, and train the logistic regression.


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

y = np.concatenate((np.zeros(corpus_train_orig_feats.shape[0]), np.ones(corpus_train_orig_feats.shape[0])))
x = np.vstack((corpus_train_orig_feats, corpus_train_shuffled_feats))

model = LogisticRegression(solver='liblinear', random_state=0).fit(x,y)



Let us now run the inference on the eval corpus.  We let sklearn do the classification, but normally here we would set prior probabilities for the task.

In [ ]:
print("Filtering using classifier")
ml_heur = model.predict(corpus_feats)
corpus = [ p for px, p in enumerate(corpus) if ml_heur[px] == 0]
print("Left with {} pairs".format(len(corpus)))